# BEAT-PD Challenge

Challenge website : https://www.synapse.org/#!Synapse:syn20825169/wiki/596118

Data information : https://www.synapse.org/#!Synapse:syn20825169/wiki/600405


### Ideas/Doubts [Laureano]

VAD like thing to remove unwanted data?
modified MFCC?
X,Y,Z = relative positions or acceleration?

Imp: Predict per person. Maybe UBM like thing and adapt it

In [ ]:
# Import required libraries

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML, display

# Imports for the high pass signal
from scipy.signal import butter, freqz, lfilter

# KFold
from sklearn.model_selection import KFold

# Import required modules
from sklearn.preprocessing import StandardScaler

import os.path

# To write WAV File
from scipy.io.wavfile import write

# To make derivative work on multiple CPUs
from concurrent.futures import ProcessPoolExecutor
from functools import partial

import sys

from sklearn.model_selection import StratifiedKFold

%load_ext autoreload
%autoreload 2

In [ ]:
# import transform_data
from transform_data import *
from create_graphs import *


In [ ]:
# Data paths
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"

# FIXME : Move this to data?
path_save_accelerometer_plots = "/home/sjoshi/codes/python/BeatPD/code/accelerometer_plots/"

possible to have participant characteristics from additional db data? ex https://ieeexplore.ieee.org/abstract/document/7911257

# CIS-PD Database

Contains 16 subject_id (patients) for the training set 

- Gender: 11 Male, 5 Female 
- Race: 15 White, 1 NA
- Ethnicity: 15 Not Hispanic or Latino, 1 Unknown
- Age average (standard deviation) : 62.8125 (10.8579)

### CIS-PD: Basic accelerometer visualization

In [ ]:
data_type = "cis"

# TODO: explain
path_train_data, df_train_label = define_data_type(data_type=data_type, data_dir=data_dir)
# display(df_train_label)
# List of interesting measurement id we want to look at
# list_measurement_id=[#'ab5287f4-8261-47ad-8ff2-22b5fe5d246e',
#'db2e053a-0fb8-4206-891a-6f079fb14e3a']#,
# 'ef5b1267-c212-46c5-aab0-4f4437bc6c67',
# '4ec74fb9-7347-435d-83dc-79ad74c3bc49',
# '8e8539ad-8841-476b-b15c-888ce3461989',
# '22b88456-fe8f-4138-af55-be12afca4b81',
# 'ad84583d-e5ae-4926-b077-531a0f7d08a9',
# 'eef56825-940a-4c3e-aebb-60838d60869e',
# 'e0441156-c4b8-467c-8f4f-3b532d594d8f',
# '464ac314-6c4b-4c4a-957c-28a2339150d6']

list_measurement_id = [
    "5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a",
    "cc7b822c-e310-46f0-a8ea-98c95fdb67a1",
    "5163afe8-a6b0-4ea4-b2ba-9b4501dd5912",
    "db2e053a-0fb8-4206-891a-6f079fb14e3a",
    "2d852742-10a9-4c56-9f38-779f2cd66879",
    "2e3a4c9c-ff01-4a28-bfcf-ce9b7633a39d",  # no inactivity should be removed
    "3cf49c01-0499-4bad-9167-67691711204a",  # no inactivity should be removed PAS LA??
    "3d0f965c-9d72-43d1-9369-1ea3acf963cc",  # PAS LA ???
    "4b269cc2-8f0c-4816-adbf-10c0069b8833",
    "4bc51b90-bfce-4231-85e1-5de3b4bc0745",
    "4fc3c295-857f-4920-8fa5-f21bfdc7ab4f",
]  # bit of inactivity in the middle]

list_measurement_id = [
    "2d852742-10a9-4c56-9f38-779f2cd66879",
    "4fc3c295-857f-4920-8fa5-f21bfdc7ab4f",
    "db2e053a-0fb8-4206-891a-6f079fb14e3a",
]

# Filter df_train_label according to the measurement_id we are most interested in
df_train_label = interesting_patients(
    df_train_label=df_train_label, list_measurement_id=list_measurement_id
)

# Display filtered df_train_label
display(df_train_label)

# path_no_inactivity_data = remove_inactivity_pct_change(df_train_label)

# Plot the accelerometer data
plot_accelerometer(df_train_label,
                   data_type=data_type,
                   path_train_data=path_train_data,
                   path_accelerometer_plots=path_save_accelerometer_plots
)

### CIS-PD: Create High Pass Data

In [ ]:
data_type = "cis"
# This is only to switch between training_data or ancillary_data which is additional data provided 
data_subset='ancillary_data' #training_data ancillary_data

path_train_data, df_train_label = define_data_type(data_type, data_dir, data_subset)

high_pass_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.'+ \
                data_subset+'.high_pass/'

high_pass_filter(df_train_label, high_pass_path, path_train_data, data_type)

### CIS-PD: Create Masks for inactivity removal 

In [ ]:
data_type = "cis"
# This is only to switch between training_data or ancillary_data which is additional data provided 
data_subset='ancillary_data' #training_data

path_train_data, df_train_label = define_data_type(data_type, data_dir, data_subset)

print(path_train_data)

In [ ]:
data_type = "cis"

remove_inactivity_highpass(
    df_train_label,
    path_train_data,
    data_type,
    energy_threshold=5,
    duration_threshold=3000,
    plot_frequency_response=False,
    mask_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.'+
    data_subset+'.high_pass_mask/')

### CIS-PD: Create first derivative

In [ ]:
data_type = "cis"
# This is only to switch between training_data or ancillary_data which is additional data provided 
data_subset='ancillary_data' #training_data

path_train_data, df_train_label = define_data_type(data_type, data_dir, data_subset)


do_work = partial(
    get_first_derivative, 
    path_train_data=path_train_data,
    derivative_path=data_dir+"cis-pd.training_data.derivative_original_data/",
    padding=True, 
    mask_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.training_data.high_pass_mask/',
)

num_jobs = 1
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

### CIS-PD: Create WAV files

#### CIS-PD: Write Wav Files - Training Data - High Pass Filter Applied

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='training_data.high_pass'
create_cis_wav_files(data_subset, data_dir, sAxis="X", data_type="cis", bMask=False)

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='training_data.high_pass'
create_cis_wav_files(data_subset, data_dir, sAxis="Y", data_type="cis", bMask=False)

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='training_data.high_pass'
create_cis_wav_files(data_subset, data_dir, sAxis="Z", data_type="cis", bMask=False)

#### CIS-PD: Write Wav Files - Training Data - Original

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='training_data'
create_cis_wav_files(data_subset, data_dir, sAxis="X", data_type="cis", bMask=False)

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='training_data'
create_cis_wav_files(data_subset, data_dir, sAxis="Y", data_type="cis", bMask=False)

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='training_data'
create_cis_wav_files(data_subset, data_dir, sAxis="Z", data_type="cis", bMask=False)

#### CIS-PD: Write Wav Files - Training Data - Inactivity Removed 

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='training_data'
create_cis_wav_files(data_subset, data_dir, sAxis="X", data_type="cis", bMask=True)

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='training_data'
create_cis_wav_files(data_subset, data_dir, sAxis="Y", data_type="cis", bMask=True)

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='training_data'
create_cis_wav_files(data_subset, data_dir, sAxis="Z", data_type="cis", bMask=True)

#### CIS-PD: Write Wav Files - Training Data - Original

In [ ]:
# Creates files in these folders:
# cis-pd.ancillary_data.wav_X
# cis-pd.ancillary_data.wav_Y
# cis-pd.ancillary_data.wav_Z

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='ancillary_data'
create_cis_wav_files(data_subset, data_dir, sAxis="X", data_type="cis", bMask=False)

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='ancillary_data'
create_cis_wav_files(data_subset, data_dir, sAxis="Y", data_type="cis", bMask=False)

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='ancillary_data'
create_cis_wav_files(data_subset, data_dir, sAxis="Z", data_type="cis", bMask=False)

#### CIS-PD: Write Wav Files - Ancillary Data - Inactivity Removed 

In [ ]:
# Creates files in these folders:
# cis-pd.ancillary_data.high_pass_mask.wav_X
# cis-pd.ancillary_data.high_pass_mask.wav_Y
# cis-pd.ancillary_data.high_pass_mask.wav_Z

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='ancillary_data'
create_cis_wav_files(data_subset, data_dir, sAxis="X", data_type="cis", bMask=True)

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='ancillary_data'
create_cis_wav_files(data_subset, data_dir, sAxis="Y", data_type="cis", bMask=True)

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='ancillary_data'
create_cis_wav_files(data_subset, data_dir, sAxis="Z", data_type="cis", bMask=True)

#### CIS-PD: Write Wav Files - Ancillary Data - High Pass Filter Applied

In [ ]:
# Creates files in these folders:
# cis-pd.ancillary_data.high_pass.wav_X
# cis-pd.ancillary_data.high_pass.wav_Y
# cis-pd.ancillary_data.high_pass.wav_Z

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='ancillary_data.high_pass'
create_cis_wav_files(data_subset, data_dir, sAxis="X", data_type="cis", bMask=False)

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='ancillary_data.high_pass'
create_cis_wav_files(data_subset, data_dir, sAxis="Y", data_type="cis", bMask=False)

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
data_subset='ancillary_data.high_pass'
create_cis_wav_files(data_subset, data_dir, sAxis="Z", data_type="cis", bMask=False)

## KFold 

In [ ]:
def get_k_fold(df_train_label,
               n_splits=5,
               subject_id=None,
               data_subset='training_data',
               data_real_subtype=""):
    """
    Function that returns a list of X dataframes (X is according to the number of n_splits chosen)

    The dataframes are the labels needed according to the split 

    Keyword Arguments:
    - df_train_label: Dataframe containing the labels
    - n_split: Optional. The number of folds. Default: 5
    - subject_id: Optional. Specify a subject_id to get measurement_id only for that subject_id 
    - data_real_subtype: Only for REAL-PD database
    """
    kf = StratifiedKFold(n_splits) # shuffle??? 

    # Building the dataframe to split
    X = []

    # if we want the data split for one specific subject_id
    if subject_id:
        df_train_label_subject_id = df_train_label.groupby("subject_id")
        X = df_train_label_subject_id.get_group(subject_id)
    else:
        # if we want to have all a split for all data no matter the subject_id
        # NOTE: I didn't make sure to have one subject_id represented in both train/test
        for idx in df_train_label.index:
            X.append([df_train_label["measurement_id"][idx]])
        X = pd.DataFrame(X)

    kf.get_n_splits(X)

    # Building lists of df_train_label because we have by default 5 splits,
    # so the lists will contain 5 DataFrames with different split indices required
    list_df_train_label = list()
    list_df_test_label = list()
    split_idx = 0
    
    # Removing NaN values only for the purpose of kf.split as stratified kfold don't like NaN values
#     print(type(X))
#     print('Before contains NaN ? : ', X.isnull().sum())
    X = prepro_missing_values(X)
#     print('After contains NaN ? : ', X_no_nan.isnull().sum())
    
    if subject_id == 1046:
        # 1046 is balanced on tremor because that's the only labels we have for that patient
        y = X.iloc[:,-1]
    else:
        # We do the KFOLDs on a balanced on/off only 
        y = X.iloc[:,-3] 

    print("----- " + str(subject_id) + " -----")
    for train_index, test_index in kf.split(X, y):
        print(len(X))
        print(len(df_train_label))
        df_train_label = X.iloc[train_index]
        df_test_label = X.iloc[test_index]
        
        # Following is just to see if the splits are balanced or not 
        train_y, test_y = y.iloc[train_index], y.iloc[test_index]
        
        train_0, train_1, train_2, train_3, train_4 = len(train_y[train_y==0]), \
                                                        len(train_y[train_y==1]), \
                                                        len(train_y[train_y==2]), \
                                                        len(train_y[train_y==3]), \
                                                        len(train_y[train_y==4])
        test_0, test_1, test_2, test_3, test_4  = len(test_y[test_y==0]), \
                                                  len(test_y[test_y==1]), \
                                                  len(test_y[test_y==2]), \
                                                  len(test_y[test_y==3]), \
                                                  len(test_y[test_y==4])
        print('>Train: 0=%d, 1=%d, 2=%d, 3=%d, 4=%d Test: 0=%d, 1=%d, 2=%d, 3=%d, 4=%d' % (train_0, train_1, train_2, train_3, train_4, \
                                                        test_0, test_1, test_2, test_3, test_4))
        
        list_df_train_label.append(df_train_label)
        list_df_test_label.append(df_test_label)

        # name of the file according to its database and type
        # NOTE: Be careful that the end of the name of the folder where to save the kfolds is hardcoded here
        path_save_k_fold_dataframes = (
            data_dir + data_type + "-pd."+data_subset+".k_fold_v2/" + data_real_subtype + "/"
        )
        df_train_label.to_csv(
            path_save_k_fold_dataframes
            + str(subject_id)
            + "_train_kfold_"
            + str(split_idx)
            + ".csv",
            index=False,
            header=["measurement_id", "subject_id", "on_off", "dyskinesia", "tremor"],
        )
        df_test_label.to_csv(
            path_save_k_fold_dataframes
            + str(subject_id)
            + "_test_kfold_"
            + str(split_idx)
            + ".csv",
            index=False,
            header=["measurement_id", "subject_id", "on_off", "dyskinesia", "tremor"],
        )
        split_idx = split_idx + 1
    return list_df_train_label, list_df_test_label

### Create the K-Fold files for the CIS database 

In [ ]:
# Define the data type as we have two databases
data_type = "cis"
path_train_data, df_train_label = define_data_type(data_type=data_type)

# Group data by subject_id
df_train_label_subject_id = df_train_label.groupby("subject_id")

data_subset = 'training_data'

# Go through the subject_id and k-fold their data
# FIXME: get_k_fold could me renamed to just create the folds, save them, not return anything
for subject_id, value in df_train_label_subject_id:
    list_df_train_label, list_df_test_label = get_k_fold(
        df_train_label=df_train_label,
        n_splits=5,
        training_or_ancillary=data_subset,
        subject_id=subject_id
    )


### Create the K-Fold Files for the REAL database

In [ ]:
# To generate the files, you have to uncomment one data_real_subtype at a time and
# execute this cell 3 times for the 3 subtypes.

data_type = "real"
data_subset='training_data' # Either training_data or ancillary_data 

for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    
    path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset,
                                                   data_real_subtype)

    # Group data by subject_id
    df_train_label_subject_id = df_train_label.groupby("subject_id")

    # Go through the subject_id and k-fold their data
    for subject_id, value in df_train_label_subject_id:
        list_df_train_label, list_df_test_label = get_k_fold(
            df_train_label=df_train_label,
            n_splits=5,
            subject_id=subject_id,
            data_subset=data_subset,
            data_real_subtype=data_real_subtype,
        )

# REAL-PD Database

This database, originally named "Parkinson@Home" is renamed to "Real-PD" for this challenge. The study was made over 2 weeks, with at home monitoring. 

The devices used are an android phone, a motorolla watch. 
- `smartwatch_accelerometer` and `smartwatch_gyroscope` : Motorolla Watch
- `smartphone_accelerometer` : Android phone 

-> Question: so is the smartwatch & smartphone accelerometer should measure the same movements? 

The REAL-PD database has many missing values. 

The subject_id `hbv013` is the only one without missing data. Other patients all have at least one missing symptom (`diskenisia`, ) or two (`on/off and tremor`, `on_off and dyskinesia`, `dyskinesia and tremor`) missing.

Measurements id with no data (`on_off`, `dyskinesia` and `tremor` are all missing):
- `b50d1b0c-2cd1-45f8-9097-0742e5cbbcc8`
- `b598c177-4e38-4ea8-8543-bd8f7e580f96`
- `cf841bf8-0082-4ea3-999f-1f43e39a8dc6`
- `b1e15f8a-109f-459b-ba87-46899240ee66`
- `6f0e2580-56ec-4743-9356-d3e4d9a0aee5`
- `773536f6-9b70-43d0-b099-5d167d74924a`
- `54a0e841-ad45-4ba7-ac83-1785c5f7748b`
- `cd9ed2e2-7e04-44c7-b041-7788f133c193`
- `a6954a91-338b-4523-9e4a-5e69a8fac206`

The 3 symptoms are reported as follows in this dataset: 
- `on_off = {0,1}`
  - `Off` : 0 (medication is wearing off) 
  - `On` : 1 (medication is working)
  
- `dyskinesia = {0,1,2}`
  - Without dyskinesia: 0 
  - Non-troublesome dyskinesia: 1 
  - Severe dyskinesia: 2 
  
- `tremor = {0,1,2,3,4}` 
The description of the database mentions `tremor` is rated from 0 to 4 according to its severity, but from all the data, the maximum value of `tremor` recorded is 3. 

Data:
- ancillary
- clinical : UPDRS evaluation score 
- demographics : #TODO


### REAL-PD: Create Masks for inactivity removal for all subtypes

In [ ]:
data_type = "real"

for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    path_train_data, df_train_label = define_data_type(data_type,
                                                       data_dir,
                                                       data_subset,
                                                       data_real_subtype)
    
    data_subset='training_data' #training_data testing_data
    
    high_pass_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.'+ \
                data_subset+'.high_pass/'+data_real_subtype+'/'
    
    high_pass_filter(df_train_label, high_pass_path, path_train_data, data_type)

In [ ]:
data_type = "real"
#data_real_subtype='smartphone_accelerometer'
#data_real_subtype = "smartwatch_accelerometer"
#data_real_subtype='smartwatch_gyroscope'

# List of interesting measurement id we want to look at
# list_measurement_id = [
#     "5b4c7c81-659d-40ea-a1fd-59622074fd10",
#     "ee053d95-c155-400d-ae42-fe24834ad4a9",
#     "ce51ee31-8553-4321-9f83-8cd3dabe2f66",
#     "e07708ff-7b8d-4070-af70-3aa81423ab5b",
#     #'7d3f4b7a-167f-4a26-9062-94ce9d8794c1',
#     "99af8d14-cd09-4107-9502-355378ba4e08",
#     #'7d5ac31a-cb53-40f7-8188-0b13724ea55c',
#     "9e43840b-dd89-498b-af1a-a62896a4d5d9",
#     "e391f546-bf8a-46c7-a16c-95bc02f40629",
# ]

# list_measurement_id = ['0c310608-1a32-4b09-b164-375d93ddb2aa']

# Filter df_train_label according to the measurement_id we are most interested in
# df_train_label = interesting_patients(df_train_label=df_train_label, list_measurement_id=list_measurement_id)
for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    data_subset='ancillary_data' #training_data testing_data 
    print(data_real_subtype)
    path_train_data, df_train_label = define_data_type(data_type,
                                                       data_dir,
                                                       data_subset,
                                                       data_real_subtype)

#     list_measurement_id = ['f465145e-562d-4c02-b347-bc42ae3b4998']#['33f5a031-43a8-496a-89ee-0b9d99019617', '2515fa8d-8320-4010-bcd9-5dba52b1b5ba']
    # Filter df_train_label according to the measurement_id we are most interested in
#     df_train_label = interesting_patients(df_train_label=df_train_label,
#                                           list_measurement_id=list_measurement_id)

    remove_inactivity_highpass(
        df_train_label,
        path_train_data=path_train_data,
        data_type=data_type,
        energy_threshold=5,
        duration_threshold=3000,
        plot_frequency_response=False,
        plot_accelerometer_after_removal=False,
        mask_path=data_dir+'/real-pd.'+
        data_subset+
        '.high_pass_mask/'+data_real_subtype+'/')

### REAL-PD: Create first derivative for all subtypes 

In [ ]:
for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    path_train_data, df_train_label = define_data_type(data_type=data_type)
    print(len(df_train_label))
    for idx in df_train_label.index:
        try:
#             print('where we get the file : ', path_train_data)
            df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
        except FileNotFoundError:
            print('Removing ' + df_train_label["measurement_id"][idx] +
                  ' as it doesn\'t exist for ' +
                  data_real_subtype)
            df_train_label = df_train_label.drop(idx)
        print(len(df_train_label))
    do_work = partial(
        get_first_derivative, 
        path_train_data=path_train_data,
        derivative_path="real-pd.training_data.derivative_original_data/"+data_real_subtype+'/',
        padding=True, 
        mask_path=data_dir+'/real-pd.training_data.high_pass_mask/'+data_real_subtype+'/'
    )

    num_jobs = 8
    with ProcessPoolExecutor(num_jobs) as ex:
        results = list(ex.map(do_work, df_train_label['measurement_id']))


### REAL-PD: Create WAV files for all subtypes

### X Axis

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="X", data_type="real")

# data_type ="real"
# #FIXME add the one missing when it's done running 
# #'smartphone_accelerometer', 'smartwatch_accelerometer', smartwatch_gyroscope
# for data_real_subtype in ['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset = 'ancillary_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type, data_subset, data_real_subtype)
# #     list_mesurement_id=['33f5a031-43a8-496a-89ee-0b9d99019617']
#     # Filter df_train_label according to the measurement_id we are most interested in
# #     df_train_label = interesting_patients(df_train_label=df_train_label, list_measurement_id=list_measurement_id)

#     for idx in df_train_label.index:
#         try:            
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)
        
#     do_work = partial(
#         write_wav, 
#         path_train_data=path_train_data,
#         wav_path=data_dir+'real-pd.'+data_subset+'.wav_X/'+data_real_subtype+'/',
#         sAxis='X',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/'
#     )

#     num_jobs = 6
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))

### Y axis

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="Y", data_type="real")


# data_type = "real"

# for data_real_subtype in ['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset='ancillary_data'#'training_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type,
#                                                    data_dir,
#                                                    data_subset,
#                                                    data_real_subtype)
#     for idx in df_train_label.index:
#         try:            
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)

#     do_work = partial(
#         write_wav,
#         path_train_data=path_train_data,
#         wav_path=data_dir+'/real-pd.'+data_subset+'.wav_Y/'+data_real_subtype+'/',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/',
#         sAxis='Y',
#     )

#     num_jobs = 8
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))

### Z Axis

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="Z", data_type="real")


# data_type = "real"

# for data_real_subtype in ['smartwatch_gyroscope']:#['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset='ancillary_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type,
#                                                    data_dir,
#                                                    data_subset,
#                                                    data_real_subtype)
#     display(df_train_label)
#     for idx in df_train_label.index:
#         try:
#             print(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)
#     display(df_train_label)
#     do_work = partial(
#         write_wav, 
#         path_train_data=path_train_data,
#         wav_path=data_dir+'/real-pd.'+data_subset+'.wav_Z/'+data_real_subtype+'/',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/',
#         sAxis='Z'
#     )

#     num_jobs = 8
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))
        

# Tests & Drafts, back-up space that's not important, just notes

In [ ]:
# Change the default option to display all row with display(DF)
# pd.set_option('display.max_rows', 1000)

### Tried to do convolution instead of np.multiply and dot product to get the derivative

In [ ]:
df2 = pd.DataFrame(
    np.array(
        [
            [0.2, 1, 5, 9],
            [0.4, 2, 6, 10],
            [0.6, 3, 7, 11],
            [0.8, 4, 8, 12],
            [1, 5, 9, 13],
            [1.2, 6, 10, 14],
            [1.4, 7, 11, 15],
            [1.6, 7, 11, 15],
            [1.8, 8, 12, 16],
            [2, 9, 13, 16],
        ]
    ),
    columns=["Timestamp", "X", "Y", "Z"],
)
display(df2)
m = np.linspace(-3, 3, num=2 * 3 + 1)
display(m)

np.convolve(df2.loc[0:6, "X"], m)

### How to filter a DataFrame

In [ ]:
# Analysis of the data to find edge cases


# Create variable with TRUE if nationality is USA
dys = df_train_label["dyskinesia"] > 1

# Create variable with TRUE if age is greater than 50
tre = df_train_label["on_off"] > 0

# Select all cases where nationality is USA and age is greater than 50
df_train_label[dys & tre]

In [ ]:
def remove_inactivity_max(df_train_label):
    last_filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    display(last_filtered_value)
    for idx in df_train_label.index:
        df_allo = []
        df_train_data = pd.read_csv(
            path_train_data + df_train_label["measurement_id"][idx] + ".csv"
        )

        # Get the absolute max value for X, Y, Z
        max_values = df_train_data.iloc[:, -3:].abs().max()

        # Compute what is 5% of that max
        thresold_energy = 5
        df_treshold = (max_values * thresold_energy) / 100

        # display(df_train_data)
        # Candidates are the frames where X, Y, Z are below that treshold (5% of the max)
        #         df_candidates = df_train_data[(df_train_data.X.abs() <= df_treshold.X) &
        #                                      (df_train_data.Y.abs() <= df_treshold.Y) &
        #                                      (df_train_data.Z.abs() <= df_treshold.Z)]
        # display(df_candidates)
        for idx2 in df_train_data.index:
            # print('df_train_data[idx2]')
            # display(df_train_data.iloc[idx2,-3:])
            last_filtered_value = filtered_value
            filtered_value = last_filtered_value + 0.004 * (
                df_train_data.iloc[idx2, -3:] - last_filtered_value
            )
            y = pd.DataFrame(columns=["Timestamp"])
            y = pd.concat(
                [y, pd.DataFrame([df_train_data.iloc[idx2, 0]], columns=["Timestamp"])],
                ignore_index=True,
            )
            #             print('display y :')
            #             display(y)
            #             print('end display y')

            #             print('display filtered value')
            #             display(pd.DataFrame(filtered_value).transpose())
            #             print('end display filtered value')
            df_allo.append(
                pd.concat([y, pd.DataFrame(filtered_value).transpose()], axis=1)
            )
        #             print('display df_allo')
        #             display(df_allo)

        # FIXME : change the name df_allo
        df_allo = pd.DataFrame(df_allo, columns=("Timestamp", "X", "Y", "Z"))

        df_allo.plot(x="Timestamp", legend=True, subplots=True, title="allo")
        stop()


#         v_candidate_x = pd.DataFrame({'Candidate':list(0)})
#         v_candidate_x = np.where(df_train_data.X.abs() <= df_treshold.X, 1, 0)

In [ ]:
# df_zeros = pd.DataFrame([False,True,False,False,False,True,False]).astype(int)

df_zeros = np.array([0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=bool)

display(df_zeros.astype(int))
count = 0
duration_threshold = 2
indices_list = []  # List of tuples
howmany = 0
for i in range(0, len(df_zeros)):
    if df_zeros[i] == 1:
        count = count + 1
        print("1 à lindex", i)
    else:
        if count >= duration_threshold:
            print("threshold atteint start ", start, " end at ", end)
            start = i - count
            end = i - 1
            indices_list.append((start, end))
            howmany = howmany + 1
            count = 0
        # if it doesn't reach the threshold, we change the 1 for 0 because we don't want to remove those
        elif count >= 1:
            print("Effacer les 1 de ", start, " a ", end)
            df_zeros[i - count : i] = [0] * (end - start)
            count = 0

display(df_zeros.astype(int))